In [1]:
import os

import pandas as pd
import numpy as np

import torch
import nltk.translate.bleu_score as bleu


from modelscope import snapshot_download
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType



2024-03-31 14:04:21,085 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-03-31 14:04:21,088 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-03-31 14:04:21,288 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 b5a2c5fe01f7460b3e700a8ce7e6fc94 and a total number of 972 components indexed
/root/miniconda3/envs/trans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
REPO_DIRECTORY = r'/root/'
ABC_DICT_PATH = r'autodl-tmp/AIST4010-Cantonese-Translator-Data/ABC-Dict/abc_dict.csv'

def load_abc_dataset():
    abc_dict = pd.read_csv(REPO_DIRECTORY + ABC_DICT_PATH)
    abc_dataset = Dataset.from_pandas(abc_dict)
    return abc_dataset

abc_set = load_abc_dataset()
abc_shuffled_set = abc_set.shuffle(seed=42).train_test_split(test_size=0.1)
abc_train_set = abc_shuffled_set['train']
abc_test_set = abc_shuffled_set['test']
for (i, example) in enumerate(abc_train_set):
    print(example)
    if i == 5:
        break

{'en': 'Today I shopped and shopped and then had spent five thousand dollars.', 'yue': '今日我買吓買吓就用咗五千蚊。'}
{'en': "Don't be so calm and unhurried. Certainly you need to buy the insurance.", 'yue': '咪咁老定，依撈七都要買保險。'}
{'en': 'In the past the two brothers were shareholders in a company, but they had conflicts every day, and there was no way they could continue cooperating with each other, so as a result they broke up and went their separate ways, and the company was then closed down.', 'yue': '兩兄弟以前係一間公司嘅股東，但係佢哋日日都有啲衝突，冇辦法合作落去，結果烏蠅摟馬尾，一拍兩散，間公司就執咗笠喇。'}
{'en': 'Try to use a bamboo stick to poke it back.', 'yue': '用竹撩佢埋嚟。'}
{'en': "They didn't have children. They feel it's troubleseome and enjoy having doggies take the place of a child.", 'yue': '佢哋冇生細路，覺得麻煩，享受以啲狗仔代替人仔。'}
{'en': 'He is very finicky about what he eats.', 'yue': '佢食嘢好㤿尖㗎。'}


In [3]:
def count_dataset_tokens(dataset):
    en_count = 0
    yue_count = 0
    for example in dataset:
        en_count += len(example['en'])
        yue_count += len(example['yue'])
    return en_count, yue_count


counts = np.array(count_dataset_tokens(abc_train_set))
print(counts)
print(counts/len(abc_train_set))


[920107 191879]
[70.63618916 14.73046215]


In [4]:
model_path=r'/root/autodl-tmp/01ai/Yi-6B-Chat'

# model = Model.from_pretrained('01ai/Yi-6B')

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype='auto'
# ).eval()


# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
model_dir = snapshot_download('01ai/Yi-6B-Chat', cache_dir='/root/autodl-tmp', revision='master')

In [6]:
base_tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, padding_side='right', max_length=512, return_tensors='pt')
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, padding_side='right', max_length=512, return_tensors='pt')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# Since transformers 4.35.0, the GPT-Q/AWQ model can be loaded using AutoModelForCausalLM.
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype='auto',
)


# # Prompt content: "hi"
# messages = [
#     {"role": "user", "content": "hi"}
# ]


# input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
# output_ids = model.generate(input_ids.to('cuda'))
# response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# # Model response: "Hello! How can I assist you today?"
# print(response)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.36s/it]


In [7]:
messages = [
    {"role": "user", "content": "hi"},
]

input_ids = base_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = base_model.generate(input_ids.to('cuda'))
response = base_tokenizer.decode(output_ids[0], skip_special_tokens=False, max_length=100)

# Model response: "Hello! How can I assist you today?"
print(response)

<|im_start|> user
hi<|im_end|> 
<|im_start|> assistant
Hello! How can I assist you today?<|im_end|>


In [8]:
# print(input_ids)
# print(output_ids)
# print(base_tokenizer.decode(input_ids[0]))
# print(base_tokenizer.decode(input_ids[0]))

# #get text of list of tokens in output_ids stored in array
# print([base_tokenizer.decode([token]) for token in output_ids[0]])

In [9]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['en'])):
        text1 = f"""<|im_start|> user
        Translate the following words into Cantonese: {example['en'][i]} <|im_end|> 
        <|im_start|> assistant
        {example['yue'][i]} <|im_end|>"""
        text2 = f"""<|im_start|> user
        Translate the following words into English:
        {example['yue'][i]} <|im_end|>
        <|im_start|> assistant
        {example['en'][i]} <|im_end|>"""
        output_texts.append(text1)
        output_texts.append(text2)
    return output_texts

In [10]:
prompts = formatting_prompts_func(abc_set[:10])
for prompt in prompts:
    print(prompt)

<|im_start|> user
        Translate the following words into Cantonese: Scoop up water <|im_end|> 
        <|im_start|> assistant
        㧾水 <|im_end|>
<|im_start|> user
        Translate the following words into English:
        㧾水 <|im_end|>
        <|im_start|> assistant
        Scoop up water <|im_end|>
<|im_start|> user
        Translate the following words into Cantonese: Ladle out soup <|im_end|> 
        <|im_start|> assistant
        㧾湯 <|im_end|>
<|im_start|> user
        Translate the following words into English:
        㧾湯 <|im_end|>
        <|im_start|> assistant
        Ladle out soup <|im_end|>
<|im_start|> user
        Translate the following words into Cantonese: Third son of a rich family <|im_end|> 
        <|im_start|> assistant
        三少 <|im_end|>
<|im_start|> user
        Translate the following words into English:
        三少 <|im_end|>
        <|im_start|> assistant
        Third son of a rich family <|im_end|>
<|im_start|> user
        Translate the following

In [11]:
# for name, param in base_model.named_parameters():
#     print(f"Parameter name: {name}")
#     print(param)
#     print("-" * 50)

In [12]:
print(base_model.config)

LlamaConfig {
  "_name_or_path": "/root/autodl-tmp/01ai/Yi-6B-Chat",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 5000000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 64000
}



In [13]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules = ["k_proj", "q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
peft_model = get_peft_model(base_model, 
                            lora_config)

peft_model.print_trainable_parameters()

trainable params: 17,825,792 || all params: 6,078,861,312 || trainable%: 0.293242288071467


**Train Tokenizer**

In [14]:
def get_training_corpus(dataset):
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        sample_en = samples["en"]
        sample_yue = samples["yue"]
        for i in range(len(sample_en)):
            yield sample_en[i]
            yield sample_yue[i]

def get_yue_training_corpus(dataset):
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        sample_yue = samples["yue"]
        for i in range(len(sample_yue)):
            yield sample_yue[i]

training_corpus = get_yue_training_corpus(abc_train_set)


curr_vocab = set(tokenizer.vocab)
# print(curr_vocab)
new_vocab = set()
#iterate through all training_corpus lines
for i, line in enumerate(training_corpus):
    unique_chars = set(list(line))
    print(unique_chars)
    new_tokens = unique_chars - curr_vocab - new_vocab
    new_vocab |= new_tokens


# for i in range(training_corpus):
#     next(training_corpus)
#     line = next(training_corpus)
#     for char in line:
#         if char not in tokenizer.vocab and char not in new_vocab:
#             new_vocab.add(char)
print(new_vocab)
print(len(new_vocab))
tokenizer.add_tokens(list(new_vocab))
tokenizer.save_pretrained("/root/tokenizer")

{'五', '我', '今', '咗', '吓', '日', '蚊', '用', '就', '千', '買', '。'}
{'七', '老', '撈', '，', '險', '咁', '咪', '依', '保', '要', '定', '都', '買', '。'}
{'蠅', '東', '辦', '果', '司', '散', '落', '突', '公', '啲', '一', '尾', '衝', '日', '笠', '兩', '作', '去', '都', '佢', '以', '執', '冇', '喇', '兄', '拍', '但', '摟', '就', '烏', '馬', '間', '弟', '結', '嘅', '，', '哋', '股', '係', '有', '咗', '合', '法', '前', '。'}
{'埋', '竹', '用', '撩', '佢', '嚟', '。'}
{'受', '仔', '代', '生', '啲', '人', '路', '享', '佢', '以', '冇', '得', '替', '覺', '煩', '細', '，', '哋', '麻', '狗', '。'}
{'尖', '㤿', '好', '食', '㗎', '嘢', '佢', '。'}
{'眼', '個', '望', '喺', '剝', '幾', '交', '生', '啲', '佬', '人', '差', '要', '打', '以', '所', '度', '同', '好', '花', '，', '我', '哋', '有', '似', '企', '。'}
{'唔', '吒', '路', '啦', '仔', '好', '。', '吱', '咁', '細'}
{'衫', '梘', '件', '吓', '先', '挼', '用', '呢', '番', '。'}
{'鐘', '分', '，', '你', '輸', '㗎', '。', '噉', '玩'}
{'米', '不', '如', '度', '老', '嘅', '，', '佢', '你', '啦', '倒', '喺', '咗', '炒', '！', '都', '細'}
{'就', '狡', '我', '真', '話', '猾', '佢', '。'}
{'唔', '歐', '本', '竹', '搵', '半', '，', '美', '個', '根

('/root/tokenizer/tokenizer_config.json',
 '/root/tokenizer/special_tokens_map.json',
 '/root/tokenizer/tokenizer.model',
 '/root/tokenizer/added_tokens.json',
 '/root/tokenizer/tokenizer.json')

In [15]:
print(tokenizer("嗌呃畀啲嘢噃")['input_ids'])
print(base_tokenizer("嗌呃畀啲嘢噃")['input_ids'])
print(tokenizer.tokenize("嗌呃畀啲嘢噃"))
print(base_tokenizer.tokenize("嗌呃畀啲嘢噃"))
print(tokenizer("Good morning")['input_ids'])
print(base_tokenizer("Good morning")['input_ids'])

[64617, 534, 450, 436, 536, 454, 433, 534, 454, 483, 534, 457, 467, 534, 458, 436]
[59568, 534, 456, 445, 534, 450, 436, 536, 454, 433, 534, 454, 483, 534, 457, 467, 534, 458, 436]
['▁嗌', '<0xE5>', '<0x91>', '<0x83>', '<0xE7>', '<0x95>', '<0x80>', '<0xE5>', '<0x95>', '<0xB2>', '<0xE5>', '<0x98>', '<0xA2>', '<0xE5>', '<0x99>', '<0x83>']
['▁', '<0xE5>', '<0x97>', '<0x8C>', '<0xE5>', '<0x91>', '<0x83>', '<0xE7>', '<0x95>', '<0x80>', '<0xE5>', '<0x95>', '<0xB2>', '<0xE5>', '<0x98>', '<0xA2>', '<0xE5>', '<0x99>', '<0x83>']
[6076, 4040]
[6076, 4040]


In [16]:
# bleu = evaluate.load('bleu')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    print(predictions.shape, labels.shape)
    return {"bleu": bleu(predictions, labels)}

In [17]:
peft_model.resize_token_embeddings(len(tokenizer))

Embedding(65170, 4096)

In [19]:
training_args = TrainingArguments(
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=3,
    logging_steps=100,
    output_dir="/root/peft_model"
)

trainer = SFTTrainer(
    peft_model,
    args=training_args,
    train_dataset= abc_train_set,
    eval_dataset= abc_test_set,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
    compute_metrics=compute_metrics,
)
trainer.train()

Map:   0%|          | 0/13026 [00:00<?, ? examples/s]

Map: 100%|██████████| 1448/1448 [00:00<00:00, 4026.51 examples/s]


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.model.save_pretrained("/root/peft_model")

/root/miniconda3/envs/trans/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/01ai/Yi-6B-Chat - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
# #get random data from test dataset
# for i in range(5):
#     example = abc_test_set[i]
#     print(example)
#     text1 = f"""Translate the following words into Cantonese: 
#         {example['en']}
#         """
#     text2 = f"""Translate the following words into English:
#         {example['yue']}
#         """
#     texts = [text1, text2]
#     for text in texts:
#         messages = [
#             {"role": "user", "content": text}
#         ]
#         print(messages)
#         #print model outputs for base_model and peft_model
#         base_input_ids = base_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
#         peft_input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
#         print("Base ID:", base_input_ids)
#         print("Base Input:", base_tokenizer.decode(base_input_ids[base_input_ids.shape[1]:], skip_special_tokens=True))
#         print("PEFT ID:", peft_input_ids)
#         print("PEFT Input:", tokenizer.decode(peft_input_ids[peft_input_ids.shape[1]:], skip_special_tokens=True))
#         print(peft_input_ids)
#         base_output_ids = base_model.generate(base_input_ids.to('cuda'), max_new_tokens=100)
#         peft_output_ids = peft_model.generate(peft_input_ids.to('cuda'), max_new_tokens=100)
#         print(base_output_ids.shape, peft_output_ids.shape)
#         print("Base model: ", base_tokenizer.decode(base_output_ids[0][base_input_ids.shape[1]:], skip_special_tokens=True))
#         print("Fine-tuned: ", tokenizer.decode(peft_output_ids[0][peft_input_ids.shape[1]:], skip_special_tokens=True))


In [ ]:
print(pd.DataFrame(trainer.state.log_history))

     loss  grad_norm  learning_rate  epoch  step  train_runtime  \
0  1.4801   0.750000       0.000667   0.03   100            NaN   
1  1.0638   0.558594       0.000333   0.06   200            NaN   
2  1.0345   0.507812       0.000000   0.09   300            NaN   
3     NaN        NaN            NaN   0.09   300        66.8764   

   train_samples_per_second  train_steps_per_second    total_flos  train_loss  
0                       NaN                     NaN           NaN         NaN  
1                       NaN                     NaN           NaN         NaN  
2                       NaN                     NaN           NaN         NaN  
3                    35.887                   4.486  8.621312e+15    1.192813  
